# 0. Cleaning the Data

- Remove all punctuation.

- Remove all non-alphanumeric characters.

- Convert all symbols to lowercase format. 

In [11]:
class Token:

    def __init__(word, f0):
        token = word
        a_count = f0
        b_count = f0
    
def data_cleaner(path: str) -> str: # author Walter and Miguel
    
    text = "" # define empty text variable
    with open(path, newline='\n') as file: # open text file and read by line
        temp_text = file.readlines() # store read lines in temporary string

    linewordlist = [] # empty list for lines, each line in this list is also a list of words
    for lineno, linedat in enumerate(temp_text): 
        linedat = linedat.strip('''!()-[]{};:'"\, <>./?@#$%^&*_~''') # take out all special trailing characters
        linedat = linedat.lower() # convert all to lowercase
        linewordlist.append(linedat.split())
    for line in linewordlist: # going through each line in the file
        for word in line: # and each word in each line
            word = word.strip("\"!()-[]{};:', <>./?@#$%^&*_~") # another strip pass to ensure all characters and whitespace are gone
            word = word.replace("'", "")
            text += word + "\n" # append word to the cleaned text string, with a newline
    # write text data to file
    file = open("cleaned.txt", "w")
    file.write(text)
    file.close()

def wordcount_dict(filepath): # author Walter
    tokendict = {} # create new empty dictionary
    with open(filepath) as file: # open file
        for line in file: # iterate through all lines (words) in the file
            line = line.strip('\n') # remove the trailing newlines
            if line not in tokendict: # check to create a new dictionary entry if token not already in dict
                tokendict.update({line : 1}) # update the dictionary with the token and only 1 since we just saw it
            elif line in tokendict: # otherwise, if it does exist
                count = tokendict.get(line) # take the existing count
                count += 1 # iterate by 1
                tokendict.update({line : count}) # update in the dictionary
    print(tokendict) # print out the dictionary


In [12]:
phase = input("Phase: ")
if phase == '1':
    path = input("Path to data file: ")
    text = data_cleaner(path)
    print(text)
if phase == '2':
    path = input("Path to cleaned text file: ")
    wordcount_dict(path)

else:
    print(F"Sorry, phase {phase} has not yet been implemented")



Phase: 2
Path to cleaned text file: cleaned.txt
{'trump': 9, 'responded': 2, 'to': 22, 'the': 42, 'boards': 9, 'ruling': 4, 'in': 11, 'a': 9, 'statement': 5, 'what': 1, 'facebook': 9, 'twitter': 3, 'and': 21, 'google': 1, 'have': 2, 'done': 1, 'is': 7, 'total': 2, 'disgrace': 1, 'an': 4, 'embarrassment': 1, 'our': 6, 'country': 3, 'free': 1, 'speech': 3, 'has': 1, 'been': 2, 'taken': 1, 'away': 1, 'from': 3, 'president': 3, 'of': 8, 'united': 1, 'states': 1, 'because': 1, 'radical': 1, 'left': 1, 'lunatics': 1, 'are': 4, 'afraid': 1, 'truth': 2, 'but': 1, 'will': 5, 'come': 1, 'out': 1, 'anyway': 1, 'bigger': 1, 'stronger': 1, 'than': 2, 'ever': 1, 'before': 1, 'people': 1, 'not': 3, 'stand': 1, 'for': 9, 'it': 5, 'these': 2, 'corrupt': 1, 'social': 4, 'media': 4, 'companies': 1, 'must': 8, 'pay': 1, 'political': 3, 'price': 1, 'never': 1, 'again': 2, 'be': 6, 'allowed': 2, 'destroy': 1, 'decimate': 1, 'electoral': 1, 'process': 1, 'separate': 1, 'released': 1, 'on': 7, 'wednesday': 2,